In [ ]:
!pip3 install pytelegrambotapi

In [ ]:
# Импортируем необходимые библиотеки
import telebot  # Для работы с Telegram API
from PIL import Image, ImageOps  # Для обработки изображений
import requests  # Для загрузки изображений по URL
from io import BytesIO  # Для работы с байтовыми данными
import numpy as np  # Для работы с массивами данных
import tensorflow as tf  # Для использования нейронной сети
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input, decode_predictions  # Модель для классификации
import pandas as pd  # Для работы с таблицами данных
from datetime import datetime  # Для работы со временем
import logging  # Для логирования ошибок

# Настройка логирования (будет записывать все ошибки в консоль)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Создаем экземпляр бота (замените 'YOUR_BOT_TOKEN' на ваш токен)
bot = telebot.TeleBot('7944203173:AAF0TyZE2DmAkSLq8RTBdV8UFtKIzcCW9rE')

# Загружаем предобученную модель EfficientNetB0 для классификации изображений
model = EfficientNetB0(weights='imagenet')

# Создаем DataFrame для хранения статистики
statistics = pd.DataFrame(columns=['Image_ID', 'Category', 'Probability', 'Upload_Time', 'Correctness'])

# Словарь для временного хранения данных пользователей
user_data = {}

# Функция для экранирования специальных символов в MarkdownV2
def escape_markdown(text):
    """
    Экранирует специальные символы для использования в MarkdownV2.
    """
    markdown_symbols = ['_', '*', '[', ']', '(', ')', '~', '`', '>', '#', '+', '-', '=', '|', '{', '}', '.', '!']
    for symbol in markdown_symbols:
        text = text.replace(symbol, f'\\{symbol}')
    return text

# Функция для классификации изображения
def classify_image(image):
    """
    Функция принимает изображение, преобразует его в формат, подходящий для модели,
    и возвращает категорию и вероятность.
    """
    image = image.resize((224, 224))  # Изменяем размер изображения до 224x224 пикселей
    image_array = np.array(image)  # Преобразуем изображение в массив NumPy
    image_array = np.expand_dims(image_array, axis=0)  # Добавляем размерность batch
    image_array = preprocess_input(image_array)  # Нормализуем значения пикселей
    predictions = model.predict(image_array)  # Получаем предсказания модели
    decoded_predictions = decode_predictions(predictions, top=1)[0]  # Выбираем топ-1 категорию
    category_name = decoded_predictions[0][1]  # Имя категории
    confidence = decoded_predictions[0][2]  # Вероятность
    return category_name, confidence

# Функция для сжатия изображения
def compress_image(image):
    """
    Функция сжимает изображение для экономии места и ускорения передачи.
    """
    compressed_image = BytesIO()  # Создаем буфер для хранения сжатого изображения
    image = ImageOps.exif_transpose(image)  # Корректируем ориентацию изображения
    image.save(compressed_image, format='JPEG', optimize=True, quality=75)  # Сохраняем изображение в JPEG
    compressed_image.seek(0)  # Возвращаем указатель в начало буфера
    return compressed_image

# Обработка команды /start
@bot.message_handler(commands=["start"])
def start(m, res=False):
    """
    Функция приветствует пользователя и объясняет, как использовать бота.
    """
    bot.send_message(m.chat.id, 'Программа по сортировке отходов запущена. Пришлите изображение.')

# Обработка изображений
@bot.message_handler(content_types=["photo"])
def handle_photo(message):
    """
    Функция обрабатывает отправленные пользователем фотографии.
    """
    try:
        # Получаем ID файла изображения
        file_id = message.photo[-1].file_id
        # Получаем информацию о файле
        file_info = bot.get_file(file_id)
        file_url = f'https://api.telegram.org/file/bot{bot.token}/{file_info.file_path}'
        # Загружаем изображение с помощью requests
        response = requests.get(file_url)

        if response.status_code == 200:  # Если изображение успешно загружено
            image = Image.open(BytesIO(response.content))  # Открываем изображение
            # Классифицируем изображение
            category_name, confidence = classify_image(image)
            # Получаем текущее время
            upload_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            # Сохраняем данные для последующего использования
            user_data[message.chat.id] = {
                'Image_ID': file_id,
                'Category': category_name,
                'Probability': confidence,
                'Upload_Time': upload_time
            }

            # Отправляем результат пользователю
            result_message = f'Категория мусора: {category_name} (вероятность: {confidence:.2f})'
            bot.send_message(message.chat.id, result_message)
            # Сжимаем и отправляем изображение обратно пользователю
            compressed_image = compress_image(image)
            bot.send_photo(message.chat.id, photo=compressed_image)

            # Запрашиваем оценку пользователя
            bot.send_message(
                message.chat.id,
                "Была ли категория определена правильно? Ответьте 'да' или 'нет'."
            )
        else:
            bot.send_message(message.chat.id, 'Не удалось загрузить изображение.')
    except Exception as e:
        # Логируем ошибку и сообщаем пользователю
        logging.error(f"Произошла ошибка: {e}")
        bot.send_message(message.chat.id, 'Произошла ошибка при обработке изображения.')

# Обработка ответов пользователя (оценка "да" или "нет")
@bot.message_handler(func=lambda message: message.chat.id in user_data and message.text.lower() in ['да', 'нет'])
def handle_feedback(message):
    """
    Функция обрабатывает ответ пользователя на вопрос о корректности определения категории.
    """
    if message.chat.id in user_data:
        # Преобразуем ответ в числовой формат (1 для "да", 0 для "нет")
        correctness = 1 if message.text.lower() == 'да' else 0

        # Достаем временные данные пользователя
        data = user_data.pop(message.chat.id)

        # Добавляем запись в таблицу статистики
        global statistics
        new_entry = pd.DataFrame({
            'Image_ID': [data['Image_ID']],
            'Category': [data['Category']],
            'Probability': [data['Probability']],
            'Upload_Time': [data['Upload_Time']],
            'Correctness': [correctness]  # Числовое значение
        })
        statistics = pd.concat([statistics, new_entry], ignore_index=True)

        # Подтверждаем получение оценки
        bot.send_message(message.chat.id, f"Спасибо за вашу оценку: {message.text.lower()}.")

# Обработка команды /help
@bot.message_handler(commands=["help"])
def help_command(message):
    """
    Функция показывает справку по использованию бота.
    """
    help_text = (
        "Доступные команды:\n"
        "/start - Начать работу с ботом\n"
        "/stop - Остановить бота\n"
        "/stats - Просмотреть статистику загрузок\n"
        "/export - Экспорт статистики в CSV\n"
        "/clearstats - Очистить статистику\n"
        "/help - Показать эту справку\n"
        "Отправьте фото, и бот определит категорию мусора. После этого вы можете оценить корректность определения, ответив 'да' или 'нет'."
    )
    bot.send_message(message.chat.id, help_text)

# Команда для просмотра статистики
@bot.message_handler(commands=["stats"])
def show_stats(message):
    """
    Функция показывает статистику загрузок изображений.
    """
    global statistics

    # Проверяем, есть ли данные в статистике
    if statistics.empty:  # Если статистика пуста
        bot.send_message(message.chat.id, "Статистика пуста.")  # Отправляем сообщение пользователю
        return  # Выходим из функции

    try:
        # Удаляем строки с пустыми значениями (NaN)
        statistics = statistics.dropna()

        # Преобразуем данные в текстовый формат
        stats_message = statistics.to_string(index=False)  # Создаем текстовое представление таблицы

        # Экранируем специальные символы для MarkdownV2
        stats_message = escape_markdown(stats_message)

        # Проверяем длину сообщения
        if len(stats_message) > 4000:  # Если сообщение слишком длинное
            # Разбиваем его на части
            chunks = [stats_message[i:i+4000] for i in range(0, len(stats_message), 4000)]
            for chunk in chunks:
                bot.send_message(message.chat.id, f"Статистика загрузок:\n\n{chunk}", parse_mode="MarkdownV2")
        else:
            # Отправляем результат пользователю
            bot.send_message(message.chat.id, f"Статистика загрузок:\n\n{stats_message}", parse_mode="MarkdownV2")

    except Exception as e:
        # Логируем ошибку для дальнейшей диагностики
        logging.error(f"Ошибка при выводе статистики: {e}")

        # Сообщаем пользователю о проблеме
        bot.send_message(message.chat.id, "Произошла ошибка при выводе статистики. Попробуйте снова позже.")

# Команда для экспорта статистики
@bot.message_handler(commands=["export"])
def export_stats(message):
    """
    Функция экспортирует статистику в CSV-файл и отправляет его пользователю.
    """
    global statistics
    if not statistics.empty:  # Если статистика не пуста
        # Создаем временный CSV-файл
        csv_buffer = BytesIO()
        statistics.to_csv(csv_buffer, index=False)
        csv_buffer.seek(0)

        # Отправляем файл пользователю
        bot.send_document(message.chat.id, csv_buffer, visible_file_name="statistics.csv")
    else:
        bot.send_message(message.chat.id, "Статистика пуста. Нечего экспортировать.")

# Команда для очистки статистики
@bot.message_handler(commands=["clearstats"])
def clear_stats(message):
    """
    Функция очищает статистику.
    """
    global statistics
    statistics = pd.DataFrame(columns=['Image_ID', 'Category', 'Probability', 'Upload_Time', 'Correctness'])  # Создаем пустую таблицу
    bot.send_message(message.chat.id, "Статистика успешно очищена.")

# Обработка остальных текстовых сообщений
@bot.message_handler(content_types=["text"])
def handle_text(message):
    """
    Функция обрабатывает остальные текстовые сообщения, которые не являются командами.
    """
    # Проверяем, является ли сообщение командой
    if message.text.startswith('/'):
        bot.send_message(message.chat.id, "Неизвестная команда. Введите /help для просмотра доступных команд.")
    else:
        bot.send_message(message.chat.id, "Неизвестное сообщение. Введите /help для просмотра доступных команд.")

# Запускаем бота
try:
    print("Бот запущен.")
    bot.polling(none_stop=True, interval=0)  # Запускаем бесконечный цикл обработки сообщений
except KeyboardInterrupt:
    print("Бот остановлен.")  # При остановке выводим сообщение